# EKF Analysis

En este análisis, se ha utilizado un Filtro de Kalman Extendido (EKF) para estimar la posición y orientación de un vehículo en un escenario específico. A continuación, se detallan los pasos y resultados obtenidos:

### Carga de Datos

Se han cargado dos archivos CSV que contienen los datos de las poses y los puntos de referencia (landmarks) del escenario. Estos archivos se han leído utilizando la librería `pandas`.

### Visualización de Datos

Se han visualizado las primeras filas y la información general de los datos de poses y landmarks para entender mejor su estructura y contenido.

### Preprocesamiento de Datos

Se han convertido las columnas `frame` y `match_index` a valores numéricos y se ha creado una nueva columna `match` en `poses_data` para indicar si hay una coincidencia con los puntos de referencia.

### Recorridos

Se han graficado las componentes X, Y y Yaw de las posiciones reales y corregidas a lo largo de los frames, destacando los puntos donde hay coincidencias con los landmarks.

### Errores de Posición y Orientación

Se han calculado los errores cuadráticos medios (MSE) para las posiciones y orientaciones estimadas en comparación con las reales.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

In [ ]:
import os
import sys

In [ ]:
# Load the files
scene = "individual_files_validation_segment-10289507859301986274_4200_000_4220_000_with_camera_labels"
date = "202504140030"

# filepath of the csv file with the scene poses_data
poses_file_path = os.path.join(os.getcwd(), scene + "/" + date, "poses_" + scene + ".csv")
landmarks_file_path = os.path.join(os.getcwd(), scene + "/" + date, "landmarks_" + scene + ".csv")
print("Poses File Path:", poses_file_path)
print("Landmarks File Path:", landmarks_file_path)


Lee información de las Poses con Pandas para visualizarlas

In [ ]:
poses_data = pd.read_csv(poses_file_path)
poses_data.shape

In [ ]:
poses_data.head()

In [ ]:
poses_data.info()

In [ ]:
real_positions = poses_data[['real_x', 'real_y']].values
estimated_positions = poses_data[['corrected_x', 'corrected_y']].values
real_orientations = poses_data[['real_yaw']].values
estimated_orientations = poses_data[['corrected_yaw']].values

Lee la informaión de las observaciones con Pandas para visualizarlas

In [ ]:
landmarks_data = pd.read_csv(landmarks_file_path)
landmarks_data.shape

In [ ]:
landmarks_data.head()

In [ ]:
landmarks_data.info()

In [ ]:
# Convertir las columnas 'frame' y 'match_index' de poses_data a valores numéricos
poses_data['frame'] = poses_data['frame'].apply(lambda x: int(x.strip('[]')))
landmarks_data['frame'] = landmarks_data['frame'].apply(lambda x: int(x.strip('[]')))
landmarks_data['match_index'] = landmarks_data['match_index'].apply(lambda x: int(x.strip('[]')))

In [ ]:
poses_data.info()

In [ ]:
landmarks_data.info()

In [ ]:
# Crear una nueva columna 'match' en poses_data
poses_data['match'] = poses_data.apply(lambda row: 1 if row['frame'] in landmarks_data[landmarks_data['match_index'] != -1]['frame'].values else 0, axis=1)

# Mostrar las primeras filas de poses_data para verificar
poses_data.head()

Recorridos

In [ ]:
plt.figure(figsize=(15, 20))

# Plot X component
plt.subplot(3, 1, 1)
plt.plot(poses_data['frame'], poses_data['real_x'], label='Real X')
plt.plot(poses_data['frame'], poses_data['corrected_x'], label='Corrected X')
plt.plot(poses_data['match'] * np.max(poses_data['real_x']), color='red', label='Match')
for i, match in enumerate(poses_data['match']):
    if match == 1:
        plt.axvline(x=poses_data['frame'][i], color='gray', linestyle='--')
plt.title('Recorrido de la Componente X')
plt.xlabel('Frame')
plt.ylabel('X (m)')
plt.legend()
plt.grid()

# Plot Y component
plt.subplot(3, 1, 2)
plt.plot(poses_data['frame'], poses_data['real_y'], label='Real Y')
plt.plot(poses_data['frame'], poses_data['corrected_y'], label='Corrected Y')
plt.plot(poses_data['match'] * np.max(poses_data['real_y']), color='red', label='Match')
for i, match in enumerate(poses_data['match']):
    if match == 1:
        plt.axvline(x=poses_data['frame'][i], color='gray', linestyle='--')
plt.title('Recorrido de la Componente Y')
plt.xlabel('Frame')
plt.ylabel('Y (m)')
plt.legend()
plt.grid()

# Plot Yaw component
plt.subplot(3, 1, 3)
plt.plot(poses_data['frame'], poses_data['real_yaw'], label='Real Yaw')
plt.plot(poses_data['frame'], poses_data['corrected_yaw'], label='Corrected Yaw')
plt.plot(poses_data['match'] * np.max(poses_data['real_yaw']), color='red', label='Match')
for i, match in enumerate(poses_data['match']):
    if match == 1:
        plt.axvline(x=poses_data['frame'][i], color='gray', linestyle='--')
plt.title('Recorrido de la Componente Yaw')
plt.xlabel('Frame')
plt.ylabel('Yaw (radianes)')
plt.legend()
plt.grid()

plt.tight_layout()
plt.show()

Errores de posición y orientación EKF

In [ ]:
mse_x = mean_squared_error(real_positions[:, 0], estimated_positions[:, 0])
mse_y = mean_squared_error(real_positions[:, 1], estimated_positions[:, 1])
mse_position_total = mean_squared_error(real_positions, estimated_positions)

print(f"MSE in position for scene {scene}: {mse_position_total:.4f}")

In [ ]:
mse_ori_yaw = mean_squared_error(real_orientations, estimated_orientations)
mse_orientation_total = mean_squared_error(real_orientations, estimated_orientations)

print(f"MSE in position for scene {scene}: {mse_orientation_total:.4f}")

In [ ]:
print("\nAnálisis de precisión (MSE):")
print(f"MSE Posición en X: {mse_x:.4f}")
print(f"MSE Posición en Y: {mse_y:.4f}")
print(f"MSE Total Posición: {mse_position_total:.4f}")

print(f"MSE Orientación en Yaw: {mse_ori_yaw:.4f}")
print(f"MSE Total Orientación: {mse_orientation_total:.4f}")

In [ ]:
position_errors = (real_positions - estimated_positions) ** 2
orientation_errors = (real_orientations - estimated_orientations) ** 2

In [ ]:
plt.figure(figsize=(15, 20))
# Plot X component
plt.subplot(3, 1, 1)
plt.plot(position_errors[:, 0], label='MSE Posición X')
plt.plot(poses_data['match'] * np.max(position_errors[:, 0]), color='red', label='Match')
plt.title('MSE de Posición X por Iteración')
plt.xlabel('Frame')
plt.ylabel('MSE')
plt.legend()
plt.grid()

# Plot Y component
plt.subplot(3, 1, 2)
plt.plot(position_errors[:, 1], label='MSE Posición Y')
plt.plot(poses_data['match'] * np.max(position_errors[:, 1]), color='red', label='Match')
plt.title('MSE de Posición Y por Iteración')
plt.xlabel('Frame')
plt.ylabel('MSE')
plt.legend()
plt.grid()

# Plot Yaw component
plt.subplot(3, 1, 3)
plt.plot(orientation_errors[:, 0], label='MSE Orientación Yaw')
plt.plot(poses_data['match'] * np.max(orientation_errors[:, 0]), color='red', label='Match')
plt.title('MSE de Orientación Yaw por Iteración')
plt.xlabel('Frame')
plt.ylabel('MSE')
plt.legend()
plt.grid()


plt.tight_layout()
plt.show()

In [ ]:
# Calcular el MSE total para cada iteración
mse_total_ekf = np.mean(position_errors, axis=1)

# Graficar el MSE total por iteración
plt.figure(figsize=(15, 5))
plt.plot(mse_total_ekf, label='MSE EKF Total')
plt.plot(poses_data['match'], color='red', label='Match')
plt.title('MSE Total por Iteración')
plt.xlabel('Iteración')
plt.ylabel('MSE Total')
plt.legend()
plt.grid()
plt.show()

## Análisis del Filtro de Kalman Extendido (EKF)

Los resultados son los siguientes:

- **MSE Posición en X:** 0.0031
- **MSE Posición en Y:** 1.1025
- **MSE Total Posición:** 0.0031
- **MSE Orientación en Yaw:** 0.0011
- **MSE Total Orientación:** 0.0011

### Análisis de Precisión

Se han graficado los errores de posición y orientación por iteración, destacando los puntos donde hay coincidencias con los landmarks.

### MSE Total por Iteración

Se ha calculado y graficado el MSE total para cada iteración, mostrando cómo varía el error a lo largo del tiempo.

### Conclusiones

El análisis muestra que el EKF es capaz de estimar la posición y orientación del vehículo con una precisión razonable. Sin embargo, se observan algunos errores significativos en ciertas iteraciones, lo que sugiere que podría haber mejoras en el modelo o en la calidad de los datos utilizados.